## 11. Alignment of Model Scales

## Numerical experiments - Week 04/2025

_Boyan Mihaylov, MSc Computational Science (UVA/VU)_

The goal of this study is to cross-verify and combine the functionalities of the high-resolution, medium-resolution and low-resolution models, in order to obtain a universal mathematical relationship between cell wall permeability, spore density and spore coverage due to clustering.

## Prerequisite libraries

In [ ]:
using PyPlot
using CurveFit
using Revise

Revise.includet("./conversions.jl")
Revise.includet("./diffusion.jl")
Revise.includet("./setup.jl")
Revise.includet("./plotting.jl")
using .Conversions
using .Diffusion
using .Setup
using .Plotting

## 1. Spore clusters in the medium and low-resolution models

The first experiment compares a high-resolution simulation of a 6-neighbour cluster with varying distances on a lattice of size $L=256\times 0.2\mu\textrm{m}$ with an absorbing boundary to a medium-resolution and a low-resolution simulation of analogical clusters on a lattice of size $L=256\times 5\mu\textrm{m}$ and $L=64\times 20\mu\textrm{m}$, respectively, with a periodic boundary. This aims to investigate the following questions:
- Does the absorbing boundary bias results compared to a free flow in a much larger space domain?
- Can the medium and low-resolution model be used for a continuation of the distance-versus-decay-exponent function for larger distances?

To this end, a range of center-to-center distances between $2R$ and $20R$ are simulated. The resolution of each model permits a specific sub-range of this set:
- high resolution: $d_s=[2R, 5R]$;
- medium resolution: $d_s=[4R, 40R]$, whereby the direct neighbourhood of spore sites is avoided;
- low resolution: $d_s=[8R, 40R]$.

In [ ]:
# Global parameters
D = 600 # microns^2/s
Ps_fit = 0.0002675353069511818 # microns/s
t_max = 14400#3600 # s
c0_spore = 1.5 # x10^(-5) M
c0_cw = 3.84 # x10^(-5) M
spore_diameter = 5 # microns
spore_rad = spore_diameter / 2.0
K = 1#316.23 # partition coefficient, set to 1 since we don't necessarily deal with lipid membranes

# Distance ranges
dist_range_hr = collect(1:0.25:2.5) * spore_diameter
dist_range_mr = collect(2:20) * spore_diameter
dist_range_lr = collect(4:4:20) * spore_diameter

# Simulation parameters
n_save_frames = 10

# High-resolution setup
N = 256
dx = 0.2
dt = 0.05
Db = Ps_fit * dx / K # compute diffusion coefficient from permeability
c_init = zeros(Float64, N, N, N)
cluster_params = [(2, true), (2, false), (6, true), (6, false)] # (neighbours, trim)

# Run high-resolution simulation
coverage_hr = zeros(Float64, length(cluster_params), length(dist_range_hr))
exponents_hr = zeros(Float64, length(cluster_params), length(dist_range_hr))
for (i, (cluster_size, trim)) in enumerate(cluster_params)
    for (j, dist) in enumerate(dist_range_hr)
        sp_cen_indices = setup_spore_cluster(cluster_size, N, 0.5 * dist / dx + 0.5, trim) # with safety radius of 0.5
        coverage_hr[i, j] = measure_coverage(sp_cen_indices[1], sp_cen_indices[2:end], spore_rad, dx)
        c_frames, c_numerical, times_numerical, region_ids, _ = diffusion_time_dependent_GPU_hi_res_implicit(copy(c_init), c0_cw, sp_cen_indices, spore_rad, t_max;
                                                                                                                D=D, Db=Db, dt=dt, dx=dx, n_save_frames=n_save_frames,
                                                                                                                crank_nicolson=false, abs_bndry=true)
        fit = exp_fit(times_numerical, c_numerical)
        exponents_hr[i, j] = fit[2]
    end
end

# Mid-resolution setup
N = 256
dx = 5
dt = 0.005
c_init = zeros(Float64, N, N, N)
cluster_sizes = [2, 3, 5, 6]

# Run mid-resolution simulation
coverage_mr = zeros(Float64, length(dist_range_mr))
exponents_mr = zeros(Float64, length(dist_range_mr))
for (i, cluster_size) in enumerate(cluster_sizes)
    for (j, dist) in enumerate(dist_range_hr)
        # Only for computation of coverage
        sp_cen_indices = setup_spore_cluster(1, N, 0.5 * dist / dx + 0.5, trim) # with safety radius of 0.5, additive single neighbour contributions
        coverage_mr[i, j] = cluster_size * measure_coverage(sp_cen_indices[1], sp_cen_indices[2], spore_rad, dx)
        c_frames, c_numerical, times_numerical, region_ids, _ = diffusion_time_dependent_GPU!(copy(c_init), c0_cw, sp_cen_indices, spore_rad, t_max;
                                                                                                D=D, Db=Db, dt=dt, dx=dx, n_save_frames=n_save_frames,
                                                                                                cluster_size=1, cluster_spacing=10)
        fit = exp_fit(times_numerical, c_numerical)
        exponents_hr[i, j] = fit[2]
    end
end

5-element Vector{Int64}:
  20
  40
  60
  80
 100

In [11]:
cluster_params = [(2, true), (2, false), (6, true), (6, false)]
length(cluster_params)

4

In [12]:
spore_idx = (0, 0, 0)
cluster_spacing = 1
spacing_combos = [-cluster_spacing, 0, cluster_spacing]
spore_cluster = vec([(i, j, k) for i in spacing_combos, j in spacing_combos, k in spacing_combos])
spore_cluster = spore_cluster[[norm(sc) for sc in spore_cluster] .== cluster_spacing]
spore_cluster = [spore_idx .+ spore for spore in spore_cluster]
order_indices = [1, 6, 2, 5, 3, 4]
# order_indices = order_indices[1:3]
spore_cluster = spore_cluster[order_indices]

6-element Vector{Tuple{Int64, Int64, Int64}}:
 (0, 0, -1)
 (0, 0, 1)
 (0, -1, 0)
 (0, 1, 0)
 (-1, 0, 0)
 (1, 0, 0)